In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pymongo
client = pymongo.MongoClient()
db = client.metrics

In [2]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging
import datetime

In [3]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [4]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [5]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-07-17 20:54:07.758742


In [6]:
CAUSER = "Causer"
RESULT = "Result"
EXPLICIT = "explicit"
CAUSER_EXPLICIT = "Causer_Explicit"
EXPLICIT_RESULT = "Explicit_Result"
CAUSER_EXPLICIT_RESULT = "Causer_Explicit_Result"
CAUSER_RESULT = "Causer_Result"

In [7]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
#TODO - don't ignore Anaphor, other and rhetoricals here
cr_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))

vtags = set(regular_tags)
#vtags.add(EMPTY_TAG)
#cr_tags = vtags

len(unique_words)

1641

In [8]:
regular_tags

['50', '5', '4', '11', '13', '3', '7', '1', '6', '14', '2', '12', '5b']

In [9]:
cr_tags

['Causer:5->Result:50',
 'Causer:7->Result:50',
 'Causer:3->Result:4',
 'Causer:13->Result:50',
 'Causer:11->Result:50',
 'Causer:1->Result:50',
 'Causer:6->Result:50',
 'Causer:3->Result:5',
 'Causer:4->Result:14',
 'Causer:3->Result:1',
 'Causer:1->Result:3',
 'Causer:3->Result:50',
 'Causer:4->Result:5',
 'Causer:3->Result:7',
 'Causer:6->Result:7',
 'Causer:11->Result:12',
 'Causer:12->Result:13',
 'Causer:7->Result:5b',
 'Causer:5b->Result:50',
 'Causer:50->Result:50',
 'Causer:1->Result:2',
 'Causer:2->Result:3',
 'Causer:11->Result:3',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:5->Result:5b',
 'Causer:1->Result:7',
 'Causer:7->Result:14',
 'Causer:5->Result:7',
 'Causer:11->Result:6',
 'Causer:13->Result:14',
 'Causer:3->Result:13',
 'Causer:3->Result:14',
 'Causer:4->Result:50',
 'Causer:14->Result:50',
 'Causer:1->Result:14',
 'Causer:4->Result:5b',
 'Causer:1->Result:4',
 'Causer:6->Result:14',
 'Causer:5b->Result:7',
 'Causer:3->Result:5b',
 'Causer:13->Resul

In [10]:
[k for k in sorted(tag_freq.keys()) if "->" not in k]

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Anaphor',
 'COMPILED',
 'Causer',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Causer:Anaphor',
 'Causer:other',
 'Causer:rhetorical',
 'Result',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7',
 'Result:Anaphor',
 'Result:other',
 'Result:rhetorical',
 'explicit',
 'other',
 'rhetorical']

## Feature Extraction

In [11]:
WINDOW_SIZE = 7

config["window_size"] = WINDOW_SIZE
offset = int((config["window_size"] - 1) / 2)
offset

3

In [12]:
def attach_function_identifier(fn, d):
    try:
        s = fn.func_name + "["
    except:
        s = fn.__name__ + "["
        fn.func_name = fn.__name__

    for k, v in sorted(d.items(), key = lambda tpl: tpl[0]):
        if k == fn.func_name:
            continue
        if type(v) == dict:
            continue
        s +=   "%s:%s " % (str(k), str(v))
    fn.func_name = s.strip() + "]"
    return fn

def fact_extract_positional_word_features_stemmed(offset):
    """ offset      :   int
                            the number of words either side of the input to extract features from
        returns     :   fn
                            feature extractor function: FeatureExtactorInput -> dict
    """
    lcls = locals()
    # curry offset
    def fn_pos_wd_feats_stemmed(input, val=1):
        return extract_positional_word_features_stemmed(offset, input, val)
    return attach_function_identifier(fn_pos_wd_feats_stemmed, lcls) # recently renamed for mongodob logging

def extract_positional_word_features_stemmed(offset, input, val = 1):
    """ offset      :   int
                           the number of words either side of the input to extract features from
        input      :    FeatureExtactorInput
                            input to feature extractor
        returns     :   dict
                            dictionary of features
    """

    feats = {}
    start = input.wordix - offset
    stop  = input.wordix + offset

    end = len(input.sentence) - 1
    for i in range(start, stop+1):
        relative_offset = str(i - input.wordix)
        if i < 0:
            feats["WD" +__START__ + ":" + relative_offset] = val
        elif i > end:
            feats["WD" +__END__ + ":" + relative_offset] = val
        else:
            offset_word = stem(input.sentence[i])
            feats["WD:" + relative_offset + "->" + offset_word] = val
    return feats

def fact_extract_ngram_features_stemmed(offset, ngram_size):
    """ offset      :   int
                            the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        returns     :   fn
                            feature extractor function: FeatureExtactorInput -> dict
    """
    lcls = locals()
    # curry offset and ngram size
    def fn_pos_ngram_feat_stemmed(input, val=1):
        return extract_ngram_features_stemmed(offset, ngram_size, input, val)
    return attach_function_identifier(fn_pos_ngram_feat_stemmed, lcls)

def extract_ngram_features_stemmed(offset, ngram_size, input, val = 1):
    """ offset      :   int
                           the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        input      :    FeatureExtactorInput
                            input to feature extractor
        returns     :   dict
                            dictionary of features
    """

    feats = {}
    end = len(input.sentence) - 1

    # fix to within bounds only
    start = max(0, input.wordix - offset)
    stop  = min(end, input.wordix + offset)

    window = list(input.sentence[start:stop+1])
    window = list(map(stem, window))
    if input.wordix < offset:
        diff = offset - input.wordix
        for i in range(diff):
            window.insert(0,__START__)
    if input.wordix + offset > end:
        diff = input.wordix + offset - end
        for i in range(diff):
            window.append(__END__)

    ngrams = compute_ngrams(window, ngram_size, ngram_size)
    str_num_ngrams = str(ngram_size)

    for i, offset_ngram in enumerate(ngrams):
        relative_offset = str(i - offset)
        str_ngram = ",".join(offset_ngram)
        feats["POS_" + str_num_ngrams + "GRAMS:" + relative_offset + "->" + str_ngram] = val

    return feats

def fact_extract_bow_ngram_features(offset, ngram_size):
    """ offset      :   int
                            the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        returns     :   fn
                            feature extractor function: FeatureExtactorInput -> dict
    """
    # curry offset and ngram size
    lcls = locals()
    def fn_bow_ngram_feat(input, val=1):
        return extract_bow_ngram_features(offset, ngram_size, input, val)
    return attach_function_identifier(fn_bow_ngram_feat, lcls)

def extract_bow_ngram_features(offset, ngram_size, input, val = 1):
    """ offset      :   int
                           the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        input      :    FeatureExtactorInput
                            input to feature extractor
        returns     :   dict
                            dictionary of features
    """

    feats = {}
    end = len(input.sentence) - 1

    # fix to within bounds only
    start = max(0, input.wordix - offset)
    stop  = min(end, input.wordix + offset)

    window = list(input.sentence[start:stop+1])
    if input.wordix < offset:
        diff = offset - input.wordix
        for i in range(diff):
            window.insert(0,__START__)
    if input.wordix + offset > end:
        diff = input.wordix + offset - end
        for i in range(diff):
            window.append(__END__)

    ngrams = compute_ngrams(window, ngram_size, ngram_size)
    str_num_ngrams = str(ngram_size)

    for i, offset_ngram in enumerate(ngrams):
        str_ngram = ",".join(offset_ngram)
        feats["POS_" + str_num_ngrams + "GRAMS:BOW" + "->" + str_ngram] = val

    return feats

In [13]:
__START__ = "<START>"
__END__   = "<END>"

#from featureextractionfunctions \
    #import fact_extract_positional_word_features_stemmed, fact_extract_ngram_features_stemmed,\
    #fact_extract_bow_ngram_features, extract_brown_cluster    
    #extract_dependency_relation

unigram_window_stemmed  = fact_extract_positional_word_features_stemmed(offset)
biigram_window_stemmed  = fact_extract_ngram_features_stemmed(offset, 2)
triigram_window_stemmed = fact_extract_ngram_features_stemmed(offset, 3)
unigram_bow_window      = fact_extract_bow_ngram_features(offset, 1)

#TODO - add in full feature set, but keep simple for now
#optimal CB feature set
extractors = [
    unigram_window_stemmed,
    biigram_window_stemmed,
    #triigram_window_stemmed,
    #unigram_bow_window,
    #extract_dependency_relation,
    #extract_brown_cluster
]

In [14]:
from Decorators import memoize
from nltk import PorterStemmer
from NgramGenerator import compute_ngrams
from featureextractortransformer import FeatureExtractorTransformer

stemmer = PorterStemmer()

@memoize
def stem(word):
    return stemmer.stem(word)

feature_extractor = FeatureExtractorTransformer(extractors)
essay_feats = feature_extractor.transform(tagged_essays)
len(tagged_essays), len(essay_feats)

(902, 902)

## Load Training Data - Essays Tagged with Codes By a Bi-Directional RNN

In [15]:
import dill
fname = predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(fname, "rb") as f:
    pred_tagged_essays = dill.load(f)

len(tagged_essays), len(pred_tagged_essays)

(902, 902)

In [16]:
def assign_predicted_tags_to_essay_feats(essay_feats, pred_tagged_essays):
    essay2pred = {}
    for essay in pred_tagged_essays:
        essay2pred[essay.name] = essay

    for essay in essay_feats:
        pred_essay = essay2pred[essay.name]
        for six, sent in enumerate(essay.sentences):
            pred_sent = pred_essay.pred_tagged_sentences[six]
            assert len(pred_sent) == len(sent), "Miss-match on sentences"
            for wix, wd in enumerate(sent):
                wd.predicted_tag = pred_sent[wix]
    
assign_predicted_tags_to_essay_feats(essay_feats, pred_tagged_essays)

## Parsing Logic

In [49]:
class Stack(object):
    def __init__(self, verbose=False):    
        self.stack = []
        self.verbose = verbose
    
    def tos(self):
        if self.len() == 0:
            return None
        #assert self.len() > 0, "Can't peek when stack is empty"
        return self.stack[-1]
    
    def pop(self):
        assert self.len() > 0, "Can't pop when stack is empty"
        item = self.stack.pop()
        if self.verbose:
            print("POPPING: %s" % item)
            print("LEN:     %i" % len(self.stack))
        return item
    
    def push(self, item):
        self.stack.append(item)
        if self.verbose:
            print("PUSHING: %s" % item)
            print("LEN:     %i" % len(self.stack))
    
    def len(self):
        return len(self.stack)

    def contains(self, item):
        return item in self.stack
    
    def __repr__(self):
        return "|".join(self.stack)
    
    def clone(self):
        clone = Stack(self.verbose)
        clone.stack = list(self.stack)
        return clone

In [50]:
ROOT = "root"

def norm_arc(arc):
    return tuple(sorted(arc))

def norm_arcs(arcs):
    return set(map(norm_arc, arcs))

class Parser(object):
    def __init__(self, stack):
        self.stack = stack
        self.arcs = []
        self.normed_arcs = set()
        # nodes with heads
        self.children = set()
        self.actions = []
        
    def get_dependencies(self):
        return [(l,r) for (l,r) in self.arcs if r != ROOT and l != ROOT]
        
    def left_arc(self, buffer):
        tos = self.stack.pop()
        #Pre-condition
        #assert self.has_head(tos) == False
        arc = (tos,buffer)
        n_arc = norm_arc(arc)
        assert n_arc not in self.normed_arcs, "Arc already processed %s" % (n_arc)
        self.arcs.append(arc)
        self.normed_arcs.add(arc)
        self.children.add(tos)
        self.actions.append("L ARC   : " + tos + "->" + buffer)
        
    def right_arc(self, buffer):
        tos = self.stack.tos()
        #normalize arc
        arc = (buffer,tos)
        n_arc = norm_arc(arc)
        assert n_arc not in self.normed_arcs, "Arc already processed %s" % (n_arc)
        self.arcs.append(arc)
        self.normed_arcs.add(n_arc)
        self.actions.append("R ARC   : " + tos + "<-" + buffer)
        self.children.add(buffer)
        self.stack.push(buffer)
        
    def reduce(self):
        tos = self.stack.pop()
        #assert self.has_head(tos) == True
        self.actions.append("REDUCE  : Pop  %s" % tos)
        
    def shift(self, buffer):
        self.stack.push(buffer)
        self.actions.append("SHIFT   : Push %s" % buffer)
    
    def skip(self, buffer):
        self.actions.append("SKIP    : item %s" % buffer)
    
    def has_head(self, item):
        return item in self.children
    
    def in_stack(self, item):
        return self.stack.contains(item)
    
    def clone(self):
        clone = Parser(self.stack.clone())
        clone.arcs = list(self.arcs)
        clone.normed_arcs = set(self.normed_arcs)
        # nodes with heads
        clone.children = set(self.children)
        clone.actions = list(self.actions)
        return clone

In [51]:
from collections import defaultdict

SHIFT = "Shift"
REDUCE = "Reduce"
LARC = "LArc"
RARC = "Rarc"
SKIP = "Skip"

class Oracle(object):
    
    def __init__(self, crels, parser):
        self.parser = parser
        self.crels = norm_arcs(crels) # type: Set[Tuple[str,str]]
        self.mapping = self.build_mappings(crels)
    
    def build_mappings(self, pairs):
        mapping = defaultdict(set)
        for c,res in pairs:
            mapping[c].add(res)
            mapping[res].add(c)
        return mapping

    def should_continue(self, action):
        # continue parsing if REDUCE or LARC
        return action in (REDUCE,LARC)
    
    def remove_relation(self, a,b):
        self.mapping[a].remove(b)
        if len(self.mapping[a]) == 0:
            del self.mapping[a]
        self.mapping[b].remove(a)
        if len(self.mapping[b]) == 0:
            del self.mapping[b]
    
    def consult(self, tos, buffer):
        """
        Performs optimal decision for parser
        If true, continue processing, else Consume Buffer
        """
        parser = self.parser
        a,b = norm_arc((tos, buffer))
        if (a,b) in self.crels:
            # TOS has arcs remaining? If so, we need RARC, else LARC
            if len(self.mapping[tos]) == 1:
                return LARC
            else:
                return RARC
        else:
            if buffer not in self.mapping:
                return SKIP
            # If the buffer has relations further down in the stack, we need to POP the TOS
            for item in self.mapping[buffer]:
                if item == tos:
                    continue
                if parser.in_stack(item):
                    return REDUCE
            #end for
            #ELSE
            return SHIFT
        
    def execute(self, action, tos, buffer):
        """
        Performs optimal decision for parser
        If true, continue processing, else Consume Buffer
        """
        parser = self.parser
        if action == LARC:
            parser.left_arc(buffer)
            self.remove_relation(tos, buffer)
        elif action == RARC:
            parser.right_arc(buffer)
            self.remove_relation(tos, buffer)
        elif action == REDUCE:
            parser.reduce()
        elif action == SHIFT:
            parser.shift(buffer)
        elif action == SKIP:
            pass
        else:
            raise Exception("Unknown parsing action %s" % action)
        return self.should_continue(action)
    
    def clone(self):
        clone = Oracle(set(self.crels), self.parser.clone())
        clone.mapping = dict(self.mapping.items())
        return clone

In [53]:
class FeatureExtractor(object):
    def __init__(self, extractors):
        self.extractors = extractors
    
    def extract(self, predicted_tag, word_seq, positive_val=1):
        """
        word: str
        word_seq: List[Word]
        
        returns: List[Dict{str,float}]
        """
        feats = []
        for seq in word_seq:
            fts = {}
            feats.append(fts)
            for ext in self.extractors:
                new_feats = ext(predicted_tag, seq, positive_val)
                fts.update(new_feats)
        return feats
    
def bag_of_word_extractor(predicted_tag, word_seq, positive_val):
    feats = {}
    for word in word_seq:
        feats["bow_" + word.word] = positive_val
    return feats

def bag_of_word_plus_tag_extractor(predicted_tag, word_seq, positive_val):
    feats = {}
    for word in word_seq:
        feats["bow_" + word.word + "_tag_" + predicted_tag] = positive_val
    return feats

feat_extractor = FeatureExtractor([
    bag_of_word_extractor,
    bag_of_word_plus_tag_extractor,
])

In [54]:
def normalize(code):
    if not code.endswith("b"):
        code += " "
    return code.rjust(3, "0")

def extract_lr(cr):
    return cr.replace("Causer:","").replace("Result:","").split("->")

def normalize_cr(cr):
    pair = extract_lr(cr)
    return (normalize(pair[0]),normalize(pair[1]))

normalize_cr("Causer:5->Result:50")

('05 ', '50 ')

In [55]:
class SearnModel(object):

    CAUSAL = set([CAUSER, EXPLICIT, RESULT])
    
    def __init__(self, feat_extractor, tags, cr_tags, base_learner_fact, beta, positive_val=1):
        # init checks
        #assert CAUSER in tags, "%s must be in tags" % CAUSER
        #assert RESULT in tags, "%s must be in tags" % RESULT
        #assert EXPLICIT in tags, "%s must be in tags" % EXPLICIT

        self.feat_extractor = feature_extractor    # feature extractor (for use later)
        self.positive_val = positive_val
        self.base_learner_fact = base_learner_fact # Sklearn classifier
        self.tags = set(tags)                      # tags for basic tagging
        
        self.cr_tags = set(cr_tags)                # causal relation tags
            
        self.actions = set([SearnModel.Shift, SearnModel.Reduce, SearnModel.Left_ARC, SearnModel.Right_ARC, SearnModel.Skip])
        self.epoch = -1
        self.initial_beta = beta
        self.beta = 1.0 # probability of using oracle for each parsing decision, initialize to 1 as we don't use until second epoch
        self.stack = []
        self.models = []
        self.training_datasets = {}
        self.current_model = None
        
    def train(self, essay_feats, epochs):
        #essay_feats = self.feat_extractor.transform(tagged_essays)        
        for i in range(0, epochs):
            self.epoch +=1
            examples_with_loss = dict() # dict of tag\decision to examples (with labels)
            new_model = self.base_learner_fact()
            
            for essay_ix, essay in enumerate(essay_feats):
                for sent_ix, taggged_sentence in enumerate(essay.sentences):
                    exs, parse = self.taggged_sentence(taggged_sentence)
                    examples.extend(exs)
            
            #TODO, dictionary vectorize examples, train a weighted binary classifier for each separate parsing action
            self.training_datasets.append(examples_with_loss)
            self.models.append(new_model)
            
            # Decay beta
            self.beta = self.beta * self.initial_beta
    
    def parse_sentence(self, taggged_sentence):
                
        #Pseudo Code
        
        # Get predicted tags
        # collapse word features for predicted tags by tag seq (gather the sequence of words assigned to that tag, they may not be contiguous)
        # compute features over these word sequences uni-grams, bi-grams, tri-grams, etc
        # include features about the buffer and the stack - tos, tos-1, buffer, buffer+1, all previous tags, all subsequent tags
        # compute features between the TOS code's word sequence and the buffer's word sequence 
            #(dep relations, pairs of words, the tag pair)
        
        # for while items in buffer:
          # read item from buffer
          # get oracle decision, if p < beta, add it to prediction_history, use as label for training data
          # get parser decision, if p >= beta add it to the prediction_history (for cond history feat extraction)
          # create new examples using features and oracle decision
          # act on decision chosen by beta threshold
       
        action_history = []
        action_tag_pair_history = []
        #x, y, weight
        examples = []
        
        arcs = set()

        all_tags = set()
        all_predicted_tags = set()
        
        min_ixs, max_ixs = defaultdict(lambda : len(taggged_sentence)+1 ), defaultdict(lambda : -1)
        ptag_seq = []
        for i, (wd) in enumerate(taggged_sentence):
            all_tags.update(wd.tags)
            ptag = wd.predicted_tag
            if ptag == EMPTY_TAG:
                continue
            if not ptag in all_predicted_tags:
                ptag_seq.append(ptag)
            all_predicted_tags.add(ptag)
            # determine span of each predicted tag
            min_ixs[ptag] = min(min_ixs[ptag], i)
            max_ixs[ptag] = max(max_ixs[ptag], i)
            
        ground_truth = all_tags.intersection(self.cr_tags)
        ground_truth = set([normalize_cr(crel) for crel in ground_truth])
        # Filter to only those crels that have support in the predicted tags
        supported_crels = set()
        for crel in ground_truth:
            l,r = crel
            if l in all_predicted_tags and r in all_predicted_tags:
                supported_crels.add(crel)
        ground_truth = supported_crels
        
        # Initialize stack, basic parser and oracle
        stack = Stack(False)        
        stack.push(ROOT)
        parser = Parser(stack)
        oracle = Oracle(ground_truth, parser)
        
        # Oracle parsing logic
        for tag_ix, ptag in enumerate(ptag_seq):
            buffer = ptag
            word_seq = taggged_sentence[min_ixs[ptag]:max_ixs[ptag]+1]
            base_feats = self.feat_extractor.extract(ptag, word_sequence, self.positive_val)
            while True:
                # make sure to include the current tag
                #TODO Get features for TOS element.
                #TODO Get interaction features between buffer feats and tos feats
                feats = self.get_conditional_feats(word_seq, action_history, action_tag_pair_history, ptag, ptag_seq[:tag_ix], ptag_seq[tag_ix+1:]) 
                feats.update(base_feats)
                
                tos = stack.tos()
                gold_action = oracle.consult(tos, buffer)
                
                # Consult Oracle or Model based on coin toss
                rand_float = np.random.random_sample() # between [0,1) (half-open interval, includes 0 but not 1)
                # If no trained models, always use Oracle
                if rand_float >= self.beta and len(self.models) > 0:
                    action = self.models[-1].predict(feats)
                else:
                    action = gold_action
                
                action_history.append(action)
                action_tag_pair_history.append((action, ptag))
                
                #TODO compute weight for each example
                #TODO - if errors are made, are there more optimal decisions?
                #TODO - if an error was made on this action, how many crels would be affected?
                    # Can we clone the current oracle and parser state, and determine the max errors if the golden action is not taken
                    # As some wrong decisions can have higher losses (e.g. a Reduce could break 2 relations where a shift may break one or be-recoverable)
                    # Then we are better training multiple binary classifiers so each decision can be weighed independently
                weight = 1
                examples.append(feats, gold_action, weight)
                
                if not oracle.execute(action, tos, buffer):
                    break
                if stack.len() == 0:
                    break
        
        return examples, action_history
            
    def get_conditional_feats(self, word_seq, action_history, action_tag_pair_history, current_tag, previous_tags, subsequent_tags):
        feats = {}
        feats["current_tag-" + current_tag]
        for i, tag in enumerate(previous_tags[::-1]):
            feats["prev_tag-{i}:{tag}".format(i=i, tag=tag)] = self.positive_val
            feats["prev_tag:{tag}".format(tag=tag)] = self.positive_val
            
        if len(previous_tags) > 0:
            bigrams = compute_ngrams(previous_tags + [current_tag], 2, 2)
            for i, bigram in enumerate(bigrams[::-1]):
                feats["prev_bigram-tag-{i}:{tag}".format(i=i, tag=str(bigram))] = self.positive_val
                feats["prev_bigram-tag:{tag}".format(tag=str(bigram))] = self.positive_val

        for i, tag in enumerate(subsequent_tags):
            feats["subseq_tag-{i}:{tag}".format(i=i, tag=tag)] = self.positive_val
        if len(subsequent_tags) > 0:
            bigrams = compute_ngrams([current_tag] + subsequent_tags, 2, 2)
            for i, bigram in enumerate(bigrams):
                feats["subseq_bigram-tag-{i}:{tag}".format(i=i, tag=str(bigram))] = self.positive_val
                feats["subseq_bigram-tag:{tag}".format(tag=str(bigram))] = self.positive_val

        action_tally = defaultdict(int)
        for i, action in enumerate(action_history[::-1]):
            feats["action-{i}:{action}".format(i=i, action=action)] = self.positive_val
            action_tally[action] +=1
        
        for action, count in action_tally.items():                           
            feats["action-tally:{action}_{count}".format(action=action, count=count)] = self.positive_val
                                       
        if len(action_history) > 0:
            bigrams = compute_ngrams(action_history + [current_tag], 2, 2)
            for i, bigram in enumerate(bigrams[::-1]):
                feats["prev_bigram_action-{i}:{tag}".format(i=i, tag=str(bigram))] = self.positive_val
                feats["prev_bigram_action:{tag}".format(tag=str(bigram))] = self.positive_val
        
        for i, (action,tag) in enumerate(action_tag_pair_history[::-1]):
            feats["actiontag-{i}:{action}_{tag}".format(i=i, action=action, tag=tag)] = self.positive_val
            feats["actiontag:{action}_{tag}".format(action=action, tag=tag)] = self.positive_val
        return feats

## Note You Can Set Per Observation Weights in XGBoost
* http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
  * see weight parameter in dtrain

## TODO
* Need to make sure the tagger tags EXCPLICT tags. These can then be skipped by the parser, but will be included in the features used to train the parser and taggger. Do we want to train a separate tagger that determines if a tagged word is a cause, explict or result. That will then resolve the direction of the relation?